# Data Wrangling

Open government data drives our research.  To get started, we'll look at three foundation datasets:

  1. Active businesses - Dataset from data.lacity.org with businesses in the city
  2. City Boundaries - Dataset from LA County containing all juristictions
  3. Opportunity Zones published by the state
  
Each of these datasets will cover multiple, common issues of open government data.

Before I start we need to set the env up.  I like to do (most) all my imports upfront.  I do it with a start.py in my profile_default.  This accomplishes the same thing.

**Note:** This can be a bit slow because it initializes osmnx.  

In [ ]:
#imports
%run start.py

# Active Businesses

Many large cities publish datasets about businesses.  LA is not exception.  They [publish](https://data.lacity.org/browse?q=Listing%20of%20Active%20Businesses&sortBy=relevance) an open dataset with businesses.  This section of code demonstrates how to [download](https://data.lacity.org/Administration-Finance/Listing-of-Active-Businesses/6rrh-rzua) directly from the site.  The following cell shows how to use pandas to read the file directly.

**Note** this is a pandas data frame.

The time to read directly from data.lacity.org will vary, but it should be a minute or so.  Since you only have to do this once, that's reasonable.

In [ ]:
%%time
biz_df = pd.read_csv('https://data.lacity.org/api/views/6rrh-rzua/rows.csv', dtype={'NAICS': str})

In [ ]:
biz_df.info()

Next step is to convert this dataframe to a geodataframe by adding a geometry attribute.  In geopandas, geometries are coded as [well known text](https://en.wikipedia.org/wiki/Well-known_text_representation_of_geometry).  Looking at the dataframe, it looks like we can use the LOCATION column.  The first issue we notice is that not all the rows have LOCATION.  As you'll see I'm also interested in NAICS codes, but that is a completely different beast!

In [ ]:
without_loc = biz_df['LOCATION'].isnull().sum()

without_naics = biz_df['NAICS'].isnull().sum()

print(f"{without_loc} ({without_loc / len(biz_df):.2f}) have no location ... a small number!")

print(f"{without_naics} ({without_naics/ len(biz_df):.2f}) have no naics ... a bigger number but ...")

For this analysis I am going to simply remove the rows with out geometry.  

I have no idea if the businesses without LOCATION are important, so I'll sketch the means to geocode using OSM. 

We can convert biz_df into those with LOCATION and those without.

In [ ]:
with_loc_biz_df = biz_df[biz_df.LOCATION.notnull()].reset_index()
without_loc_biz_df = biz_df[biz_df.LOCATION.isnull()].reset_index()

You can verify the two dataframes have the desired LOCATION properties.

## Geocoding

Now we have the simple example geocoding function we'll use.

**Note the caveats**

In [ ]:
def geocode_address(row):
    """
    Example function to geocode.
    Uses the nominatim server from osm.
    Should build in a one second pause as per the terms of nominatim.
    Not adding any type of error handling...
    """
    zip5 = row['ZIP CODE'].split('-')[0]
    geocode_query = f"{row['STREET ADDRESS']} {row['CITY']} CA {zip5}"
    lat, lon = ox.geocode(geocode_query)
    return Point(lon, lat)

With this function, we can demonstrate simple geocoding.  The next steps:

  1. Examine the dataframe without LOCATION
  2. Look at one row from the dataframe
  3. Use that row as the argument to our geocoding function
  
I did not turn the ox.gecode information output, so you can see that we get a lat/long.

Next, let's look at a value with no LOCATION and use that for geocode function.

In [ ]:
without_loc_biz_df.iloc[0]

In [ ]:
geocode_address(without_loc_biz_df.iloc[0])

Or, if you want to see the wkt point.

**Note:** If you try this with other points in the df you may get nominatim errors.  See the workshop repo for more details.

In [ ]:
_.wkt

## NAICS

I'm adding a preprocessing step on the [NAICS](https://en.wikipedia.org/wiki/North_American_Industry_Classification_System) codes.  This enables higher level queries on business types when we get into the analysis phase.  NAICS codes add complexity.  They have been used since the 90's so there are multiple versions.  For purposes of most business datasets they are self applied.  Messy!

For this dataset I want to add a business sector description to each row (if it has a valid NAICS).  I'm using definitions from 2017.  There is a new verion but ...

The steps:

  1. Use xls from [Census](https://www.census.gov/naics/) to get the mappings
  2. Slice the first two NAICS characters (business sector) from the businesses dataset
  3. Lookup/add the description from the xls-based dictionary
  
Hacky, fast and dirty.  For more you can visit [previous workshop](https://github.com/researchsherpa/workshop).

In [ ]:
naics_desc = pd.read_excel('../data/2017_NAICS_Descriptions.xls')

naics_desc['Code'] = naics_desc['Code'].astype(str)

lookup_dict = naics_desc[['Code', 'Title']].set_index('Code')['Title'].to_dict()

def lookup(key):
    """
    Serious hack!
    Modified the xls to reflect values in the data.
    Object is understanding at the business sector level as query/search.
    """
    
    if key in lookup_dict.keys():

        desc = lookup_dict[key]
        if desc[-1:] == 'T':
            desc = desc[:-1]
    else:
        desc = 'No NAICS'
        
    return desc.rstrip()

In [ ]:
with_loc_biz_df['sector'] = with_loc_biz_df['NAICS'].astype(str).apply(lambda x: x[:2])

with_loc_biz_df['sector_desc'] = with_loc_biz_df['sector'].apply(lambda x: lookup(x) + f"({x})")

Now we can look at the distribution, by business sector, for businesses in LA.

**Note:** I won't deal with missing or bad codes.

In [ ]:
with_loc_biz_df.sector_desc.value_counts()

## Geodataframe

The final step is to convert the pandas dataframe into a geopandas geodataframe.  The key step is adding a geometry column, using LOCATION.

As usual there are some **issues/warnings** with shapely projections, proj, and geopandas.  Since the versions I'm using for each of these packages are just showing the warnings, I'm not going to change things.  As upgrades happen this will have to be revisited.  For what we're doing now it doesn't matter.

In [ ]:
def location_to_wkt(row_location):
    """
    Utility function to convert the str based LOCATION into wkt.
    wkt is the x-y version of lat-long
    Note that longitude is the x
    """
    x, y = np.array(row_location.strip('(').strip(')').replace(',', '').split(' ')[::-1])
    return Point(float(x), float(y))
    

In [ ]:
with_loc_biz_gdf = gpd.GeoDataFrame(with_loc_biz_df,
                                crs={'init': 'epsg:4326'},
                                #crs='+init=epsg:4326',
                                geometry=with_loc_biz_df.apply(lambda row: location_to_wkt(row.LOCATION), axis=1))

Now info() shows the geodataframe with geometry column.  This is what we need to map the data and for any spatial processing.

In [ ]:
with_loc_biz_gdf.info()

In [ ]:
with_loc_biz_gdf = with_loc_biz_gdf.to_crs("EPSG:4326")

In [ ]:
with_loc_biz_gdf.crs

## Save the geodataframe

Final step is to save the newly minted geopandas dataframe.  I'm using the **parquet file** format even though we get the **warning** from geopandas.  I have not had any problems with their version and it is way faster!

In [ ]:
with_loc_biz_gdf.to_parquet('../data/businesses-gdf.parq')

So at this stage we have the buinesses dataset in a format for some geoanalysis.

# LA Cities

This is the first of two datasets with polygon geometries we'll add to the analysis.

The dataset is published by the Los Angeles County.  It includes boundaries for the different jurisdictions in the county, incorporated and unincorporated.

What I want for this analysis is the boundary (polygon) for the City of Los Angeles.

In [ ]:
IFrame("https://egis-lacounty.hub.arcgis.com/datasets/la-county-city-boundaries/explore?location=33.777536%2C-118.302300%2C9.00", width=1200, height=800)

For this dataset I've downloaded the shape file as a zip.  Geopandas is smart enough to handle it directly.

In [ ]:
city_boundaries_gdf = gpd.read_file('../data/LA_County_City_Boundaries.zip')

Let's look at the crs for this dataset.  Note the analysis is typically done with epsg:4326.

In [ ]:
city_boundaries_gdf.crs

If you want to know more about the crs, look at this:

In [ ]:
IFrame('https://epsg.io/3857', width=1200, height=800)

Let's change this while I'm thinking about it.

In [ ]:
city_boundaries_gdf.to_crs('EPSG:4326', inplace=True)

So this is a collection of polygons for all the jurisdictions in LA County.  

geopandas has an easy way to visualize the data using folium:

In [ ]:
city_boundaries_gdf[['CITY_NAME', 'geometry']].explore(
     column="CITY_NAME", 
     tooltip="CITY_NAME", 
     popup=True, 
     tiles="CartoDB positron", 
     cmap="Set1", 
     style_kwds=dict(color="black") 
    )

How can we use this to build a geodataframe with one polygon containing the boundary for the City of LA?  For that, let's dig a bit deeper.  Using two attributes, CITY_NAME and FEAT_TYPE, will get us closer.

In [ ]:
city_boundaries_gdf.CITY_NAME.value_counts()

In [ ]:
city_boundaries_gdf.FEAT_TYPE.value_counts()

In [ ]:
la_boundary_gdf = city_boundaries_gdf.query(f"CITY_NAME == 'Los Angeles' and FEAT_TYPE == 'Land'").reset_index().drop(columns=['index'])

In [ ]:
la_boundary_gdf

There are six `land features` for City of LA.  I will not spend time to figure why these features are used, what is different, where they came from, ...  I just want a row with the boundary polygon (could be duplicated?).

Look at the first row:

In [ ]:
la_boundary_gdf.loc[:0].explore()

Good enough.  Let's save it for upstream processing.

In [ ]:
la_boundary_gdf.loc[:0].to_parquet('../data/la-boundary.parq')

# Opportunity Zones

Finally, for now, we'll look at [opportunity zones](https://en.wikipedia.org/wiki/Opportunity_zone).  They are Census tracts that meet certain criteria.  The State of California is responsible for which tracts to include.

In [ ]:
IFrame("https://dof.ca.gov/forecasting/demographics/california-opportunity-zones/", width=1200, height=800)

The shape file has been downloaded for use here.

In [ ]:
opportunity_zones_gdf = gpd.read_file('../data/FnlOZTracts.zip')

In [ ]:
opportunity_zones_gdf.explore()

By now you should know the drill.  Make sure we have the standard crs and save as parquet file.

In [ ]:
opportunity_zones_gdf.crs

In [ ]:
opportunity_zones_gdf.to_crs('EPSG:4326', inplace=True)

In [ ]:
opportunity_zones_gdf.to_parquet('../data/opportunity-zones.parq')

# Conclusion

Three datasets have been curated for the rest of the analysis.  

  1. Active businesses published by the City of Los Angeles
  2. LA City boundary obtained from county jurisdictions data
  3. Opportunity zone polygons from the State of California
  
All the files are in the ../data directory stored as parquet files (*.parq).